# AML Mini-Challenge - Credit Card Affinity Modelling

> Dominik Filliger & Noah Leuenberger

The task can be found [here](https://spaces.technik.fhnw.ch/storage/uploads/spaces/82/exercises/20240218__AML_Trainingscenter_MiniChallenge_Kreditkarten_Aufgabenstellung-1708412668.pdf).

# Setup

In [ ]:
import pandas as pd
from datetime import datetime

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()
#plt.style.use('seaborn-white')
#plt.style.use('ggplot')

## Helper Functions

In [ ]:
def plot_categorical_variables(df, categorical_columns, fill_na_value='NA'):
    """
    Plots count plots for categorical variables in a DataFrame, filling NA values with a specified string.
    
    Parameters:
    - df: pandas.DataFrame containing the data.
    - categorical_vars: list of strings, names of the categorical variables in df to plot.
    - fill_na_value: string, the value to use for filling NA values in the categorical variables.
    """
    # Fill NA values in the specified categorical variables
    for var in categorical_columns:
        if df[var].isna().any():
            df[var] = df[var].fillna(fill_na_value)

    total = float(len(df))
    fig, axes = plt.subplots(nrows=len(categorical_columns), figsize=(14, len(categorical_columns) * 4.5))

    if len(categorical_columns) == 1:  # If there's only one categorical variable, wrap axes in a list
        axes = [axes]

    for i, var in enumerate(categorical_columns):
        ax = sns.countplot(x=var, data=df, ax=axes[i], order=df[var].value_counts().index)

        axes[i].set_title(f'Distribution of {var}')
        axes[i].set_ylabel('Count')
        axes[i].set_xlabel(var)

        for p in ax.patches:
            height = p.get_height()
            ax.text(p.get_x() + p.get_width() / 2.,
                    height + 3,
                    '{:1.2f}%'.format((height / total) * 100),
                    ha="center")

    plt.tight_layout()
    plt.show()

def plot_numerical_distributions(df, numerical_columns, kde=True, bins=30):
    """
    Plots the distribution of all numerical variables in a DataFrame.
    
    Parameters:
    - df: pandas.DataFrame containing the data.
    """

    # Determine the number of rows needed for subplots based on the number of numerical variables
    nrows = len(numerical_columns)

    # Create subplots
    fig, axes = plt.subplots(nrows=nrows, ncols=1, figsize=(8, 5 * nrows))

    if nrows == 1:  # If there's only one numerical variable, wrap axes in a list
        axes = [axes]

    for i, var in enumerate(numerical_columns):
        sns.histplot(df[var], ax=axes[i], kde=kde, bins=bins)
        axes[i].set_title(f'Distribution of {var}')
        axes[i].set_xlabel(var)
        axes[i].set_ylabel('Frequency')

    plt.tight_layout()
    plt.show()

def plot_date_monthly_counts(df, date_column, title):
    """
    Plots the monthly counts of a date column in a DataFrame.
    
    Parameters:
    - df: pandas.DataFrame containing the data.
    - date_column: string, name of the date column in df to plot.
    - title: string, title of the plot.
    """
    df[date_column] = pd.to_datetime(df[date_column])
    df['month'] = df[date_column].dt.to_period('M')

    monthly_counts = df['month'].value_counts().sort_index()
    monthly_counts.plot(kind='bar', figsize=(14, 6))
    plt.title(title)
    plt.xlabel('Month')
    plt.ylabel('Count')
    plt.show()

def add_percentage_labels(ax, hue_order):
    for p in ax.patches:
        height = p.get_height()
        width = p.get_width()
        x = p.get_x()
        y = p.get_y()
        label_text = f'{height:.1f}%'
        label_x = x + width / 2
        label_y = y + height / 2
        ax.text(label_x, label_y, label_text, ha='center', va='center', fontsize=9, color='white', weight='bold')

In [ ]:
from collections import OrderedDict
data_reduction = OrderedDict()

# Data Import & Wrangling

In [ ]:
def remap_values(df, column, mapping):
    # assert that all values in the column are in the mapping except for NaN
    assert df[column].dropna().isin(mapping.keys()).all()
    
    df[column] = df[column].map(mapping, na_action='ignore')
    return df

def map_empty_to_nan(df, column):
    if df[column].dtype != 'object':
        return df

    df[column] = df[column].replace(r'^\s*$', np.nan, regex=True)
    return df

def read_csv(file_path, sep=";", dtypes=None):
    df = pd.read_csv(file_path, sep=sep, dtype=dtypes)
    
    for col in df.columns:
        df = map_empty_to_nan(df, col)
        
    return df

## Accounts

In [ ]:
accounts_df = read_csv("data/account.csv")

# Translated frequency from Czech to English
# according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
accounts_df = remap_values(accounts_df, 'frequency', {
    "POPLATEK MESICNE": "MONTHLY_ISSUANCE",
    "POPLATEK TYDNE": "WEEKLY_ISSUANCE",
    "POPLATEK PO OBRATU": "ISSUANCE_AFTER_TRANSACTION"
})

accounts_df['date'] = pd.to_datetime(accounts_df['date'], format='%y%m%d')

accounts_df.rename(columns={'date': 'account_created',
                         'frequency': 'account_frequency'}, inplace=True)

data_reduction["Total number of accounts"] = len(accounts_df)
accounts_df.info()

In [ ]:
# todo add some basic eda here
accounts_df.head()

In [ ]:
accounts_df.nunique()

In [ ]:
plot_categorical_variables(accounts_df, ['account_frequency'])

In [ ]:
plot_numerical_distributions(accounts_df, ['account_created'])

## Clients

In [ ]:
clients_df = read_csv("data/client.csv")

def parse_birth_number(birth_number):
    birth_number_str = str(birth_number)

    # Extract year, month, and day from birth number from string
    # according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
    year = int(birth_number_str[:2])
    month = int(birth_number_str[2:4])
    day = int(birth_number_str[4:6])

    # Determine sex based on month and adjust month for female clients
    # according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
    if month > 50:
        sex = "Female"
        month -= 50
    else:
        sex = "Male"

    # Validate date
    assert 1 <= month <= 12
    assert 1 <= day <= 31
    assert 0 <= year <= 99

    if month in [4, 6, 9, 11]:
        assert 1 <= day <= 30
    elif month == 2:
        assert 1 <= day <= 29
    else:
        assert 1 <= day <= 31

    # Assuming all dates are in the 1900s
    birth_date = datetime(1900 + year, month, day)
    return pd.Series([sex, birth_date])


clients_df[['sex', 'birth_date']] = clients_df['birth_number'].apply(parse_birth_number)

# Calculate 'age' assuming the reference year is 1999
clients_df['age'] = clients_df['birth_date'].apply(lambda x: 1999 - x.year)

# Drop 'birth_number' column as it is no longer needed
clients_df = clients_df.drop(columns=['birth_number'])

clients_df.info()

In [ ]:
# todo add some basic eda here
clients_df.head()

In [ ]:
clients_df.describe()

In [ ]:
plot_numerical_distributions(clients_df, ['birth_date', 'age'])

## Dispositions

In [ ]:
dispositions_df = read_csv("data/disp.csv")
dispositions_df.info()

In [ ]:
dispositions_df.head()

In [ ]:
dispositions_df.describe()

In [ ]:
plot_categorical_variables(dispositions_df, ['type'])

As the goal of this model is to address accounts and not client directly we will focus on the clients which own an account and focus solely on them. 

In [ ]:
dispositions_df = dispositions_df[dispositions_df['type'] == 'OWNER']

## Orders

In [ ]:
orders_df = read_csv("data/order.csv")

# Translated from Czech to English
# according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
orders_df = remap_values(orders_df, 'k_symbol', {
    "POJISTNE": "Insurance_Payment",
    "SIPO": "Household",
    "LEASING": "Leasing",
    "UVER": "Loan_Payment"
})

orders_df['account_to'] = orders_df['account_to'].astype('category')

orders_df = orders_df.rename(columns={'amount': 'debited_amount'})

orders_df.info()

In [ ]:
orders_df.head()


In [ ]:
orders_df.describe()

In [ ]:
orders_df.nunique()

There appear to be as many order ids as there are rows. 

In [ ]:
plot_categorical_variables(orders_df, ['k_symbol', 'bank_to'])

In [ ]:
plot_numerical_distributions(orders_df, ['debited_amount'])

## Transactions

In [ ]:
# column 8 is the 'bank' column which contains NaNs and must be read as string
transactions_df = read_csv("data/trans.csv", dtypes={8: str})

transactions_df['date'] = pd.to_datetime(transactions_df['date'], format='%y%m%d')

# Translated type, operations and characteristics from Czech to English
# according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
transactions_df = remap_values(transactions_df, 'type', {
    "VYBER": "Withdrawal", # Also withdrawal as it is against the documentation present in the dataset
    "PRIJEM": "Credit",
    "VYDAJ": "Withdrawal"
})

transactions_df = remap_values(transactions_df, 'operation', {
    "VYBER KARTOU": "Credit Card Withdrawal",
    "VKLAD": "Credit in Cash",
    "PREVOD Z UCTU": "Collection from Another Bank",
    "VYBER": "Withdrawal in Cash",
    "PREVOD NA UCET": "Remittance to Another Bank"
})

transactions_df = remap_values(transactions_df, 'k_symbol', {
    "POJISTNE": "Insurance Payment",
    "SLUZBY": "Payment on Statement",
    "UROK": "Interest Credited",
    "SANKC. UROK": "Sanction Interest",
    "SIPO": "Household",
    "DUCHOD": "Old-age Pension",
    "UVER": "Loan Payment"
})

# Set the amount to negative for withdrawals and positive for credits
transactions_df['amount'] = np.where(transactions_df['type'] == "Credit", transactions_df['amount'], -transactions_df['amount'])

transactions_df.rename(columns={'type': 'transaction_type'}, inplace=True)

transactions_df.info()

In [ ]:
transactions_df.head()

In [ ]:
transactions_df.describe()

In [ ]:
plot_categorical_variables(transactions_df, ['transaction_type', 'operation', 'k_symbol'])

In [ ]:
plot_numerical_distributions(transactions_df, ['date', 'amount', 'balance'])

Looking at the distributions of the transaction table we can see that the count of transactions per year increase over time. So we can conclude that the bank has a growing client base. 

However, the other plots are not very useful. For one the transaction amount seems to be very sparse, ranging from values between -80000 and 80000. 

The balance distribution also showcases that there are accounts with a negative balance after a transaction, which would only make sense if debt is also included in this value. 

According to description of the field balance: "balance after transaction"

In [ ]:
# Getting a list of unique years from the dataset
transactions_df['year'] = transactions_df['date'].dt.year
transactions_df['month'] = transactions_df['date'].dt.month

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


years = sorted(transactions_df['year'].unique())

# Creating a figure with subplots for each year: one row for each year with two plots (box plot and bar chart)
fig, axs = plt.subplots(len(years) * 2, 1, figsize=(15, 6 * len(years)), sharex=True, gridspec_kw={'height_ratios': [3, 1] * len(years)})

for i, year in enumerate(years):
    # Filter transactions for the current year
    yearly_transactions = transactions_df[transactions_df['year'] == year]

    # Preparing data for the box plot: a list of amounts for each month for the current year
    amounts_per_month_yearly = [yearly_transactions[yearly_transactions['month'] == month]['amount'] for month in range(1, 13)]

    # Preparing data for the bar chart for the current year
    monthly_summary_yearly = yearly_transactions.groupby('month').agg(TotalAmount=('amount', 'sum'), TransactionCount=('amount', 'count')).reset_index()

    # Box plot for transaction amounts by month for the current year
    axs[i*2].boxplot(amounts_per_month_yearly, patch_artist=True)
    # now with seaborn
    # sns.boxplot(data=yearly_transactions, x='month', y='amount', ax=axs[i*2])
    axs[i*2].set_title(f'Transaction Amounts Per Month in {year} (Box Plot)')
    axs[i*2].set_yscale('symlog')
    axs[i*2].set_ylabel('Transaction Amounts (log scale)')
    axs[i*2].grid(True, which='both')

    # Bar chart for transaction count by month for the current year
    axs[i*2 + 1].bar(monthly_summary_yearly['month'], monthly_summary_yearly['TransactionCount'], color='tab:red', alpha=0.6)
    axs[i*2 + 1].set_ylabel('Transaction Count')
    axs[i*2 + 1].grid(True, which='both')

# Setting x-ticks and labels for the last bar chart (shared x-axis for all)
axs[-1].set_xticks(range(1, 13))
axs[-1].set_xticklabels(months)
axs[-1].set_xlabel('Month')

plt.tight_layout()
plt.show()



In [ ]:
# Importing required libraries for visualization
import matplotlib.pyplot as plt
import pandas as pd

# Assuming 'transactions' DataFrame already contains 'year' and 'month' columns
# and is ready for visualization

# Getting a list of unique years and defining month labels
years = sorted(transactions_df['year'].unique())
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Adjusting the figure layout to place visualizations for each year next to each other
fig, axs = plt.subplots(2, len(years), figsize=(40 * len(years) / 2, 30), sharey='row', gridspec_kw={'height_ratios': [3, 1]})

for i, year in enumerate(years):
    # Filter transactions for the current year
    yearly_transactions = transactions_df[transactions_df['year'] == year]

    # Preparing data for the box plot: a list of amounts for each month for the current year
    amounts_per_month_yearly = [yearly_transactions[yearly_transactions['month'] == month]['amount'] for month in range(1, 13)]

    # Preparing data for the bar chart for the current year
    monthly_summary_yearly = yearly_transactions.groupby('month').agg(TotalAmount=('amount', 'sum'), TransactionCount=('amount', 'count')).reset_index()

    # Selecting the appropriate axes for multiple or single year scenarios
    ax_box = axs[0, i] if len(years) > 1 else axs[0]
    ax_bar = axs[1, i] if len(years) > 1 else axs[1]
    
    ax_box.boxplot(amounts_per_month_yearly, patch_artist=True)
    ax_box.set_title(f'{year} (Box Plot)')
    ax_box.set_yscale('symlog')
    ax_box.set_ylabel('Transaction Amounts (log scale)')
    ax_box.grid(True, which='both')

    ax_bar.bar(monthly_summary_yearly['month'], monthly_summary_yearly['TransactionCount'], color='tab:red', alpha=0.6)
    ax_bar.set_ylabel('Transaction Count')
    ax_bar.grid(True, which='both')

    # Setting common x-ticks and labels for all axes
    ax_bar.set_xticks(range(1, 13))
    ax_bar.set_xticklabels(months)

# Setting a common x-label for the entire figure
fig.text(0.5, 0.04, 'Month', ha='center')

plt.tight_layout()
plt.show()


In [ ]:
negative_balances = transactions_df[transactions_df['balance'] < 0]
plot_numerical_distributions(negative_balances, ['balance', 'amount'])
print(f"Number of transactions with negative balance: {len(negative_balances)}")

There appear to be 2999 transactions which have a negative balance, therefore after the transaction the account balance was negative. This implies that these accounts are in some kind of debt. 

## Loans

In [ ]:
loans_df = read_csv("data/loan.csv")

loans_df['date'] = pd.to_datetime(loans_df['date'], format='%y%m%d')

loans_df['status'] = loans_df['status'].map({
    "A": "Contract finished, no problems",
    "B": "Contract finished, loan not paid",
    "C": "Contract running, OK thus-far",
    "D": "Contract running, client in debt"
})

loans_df.rename(columns={
    'date': 'granted_date',
    'amount': 'amount',
    'duration': 'duration',
    'payments': 'monthly_payments',
    'status': 'status'
}, inplace=True)

loans_df.info()

In [ ]:
# todo add some basic eda here
loans_df.head()


In [ ]:
loans_df.describe()


In [ ]:
loans_df.nunique()

It seems as if one account can have at max one loan.

In [ ]:
plot_categorical_variables(loans_df, ['duration', 'status'])


The distribution of durations seems to be even.

In [ ]:
plot_numerical_distributions(loans_df, ['granted_date'])


## Credit Cards

In [ ]:
cards_df = read_csv("data/card.csv")

cards_df['issued'] = pd.to_datetime(cards_df['issued'], format='%y%m%d %H:%M:%S').dt.date

cards_df.info()

In [ ]:
cards_df.head()

In [ ]:
cards_df.describe()

In [ ]:
plot_categorical_variables(cards_df, ['type'])

In [ ]:
plot_numerical_distributions(cards_df, ['issued'])

## Demographic data

In [ ]:
districts_df = read_csv("data/district.csv")

# Rename columns
# according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
districts_df.rename(columns={
    'A1': 'district_id',
    'A2': 'district_name',
    'A3': 'region',
    'A4': 'inhabitants',
    'A5': 'small_municipalities',
    'A6': 'medium_municipalities',
    'A7': 'large_municipalities',
    'A8': 'huge_municipalities',
    'A9': 'cities',
    'A10': 'ratio_urban_inhabitants',
    'A11': 'average_salary',
    'A12': 'unemployment_rate_1995',
    'A13': 'unemployment_rate_1996',
    'A14': 'entrepreneurs_per_1000_inhabitants',
    'A15': 'crimes_committed_1995',
    'A16': 'crimes_committed_1996'
}, inplace=True)

for col in ['unemployment_rate_1995', 'unemployment_rate_1996', 'crimes_committed_1995', 'crimes_committed_1996']:
    districts_df[col] = pd.to_numeric(districts_df[col], errors='coerce')

districts_df.info()

It appears as if there is 1 null value for unemployment rate in 1995 and crimes committed in 1995.

In [ ]:
# todo add some basic eda here
districts_df.head()

In [ ]:
districts_df.describe()

In [ ]:
districts_df.nunique()

In [ ]:
plot_numerical_distributions(districts_df, ['crimes_committed_1995'])

In [ ]:
plot_categorical_variables(districts_df, ['region'])

We need to differentiate between the domicile of the client and account, as they can be different.


# Relationships

Following the documentation of the dataset, there are multiple relationships that need to be validated. https://sorry.vse.cz/~berka/challenge/PAST/index.html

The ERD according to the descriptions on https://sorry.vse.cz/~berka/challenge/PAST/index.html

[![](https://mermaid.ink/img/pako:eNqtV1Fv4jgQ_itWXu6l7SZ0gQatTsqGdhddCxVQrXSqFJnEgLWJnbOd7bGl__3GTgA3JGyvWh7ajPPNZ898nrHz7MQ8Ic7AIWJI8Urg7JEh-AVhOHkYz9FzaeqfVIKyFcJxzAumIpqg-7_QoxOUNhoNH50jcEL1Q2zQNxo9rAZq8AQrUv45EIaCYEU5Q0MYb-BeCvJPQVi8AZ-RlAVmMUE3u8Gdw0v5L7wdXTeGE6eUWNGExmxa3YIKtY5YkS2IAOBnbbYt7S1hVwsbjmb3k9loPpqMG1YHRPl-bUCSc0lNThqzfYjlpjWWYxVvfqmi2uRamMkTI-LDgySiFsNkOryeNqyei4SI_fIn2voNq1lg9j1SHGBTEtPcxPgZxk7w1tAVve2wTDlWCGdmWkg1WVBFEhRkdWDF_D2Sm2zBU8De402mWcM1FjhWRNCfZt_WkjSfBuNZELYIrQRmcp-qubaAq1XptyXMriqbsmXXVjJD3SVUffhG1ToR-AmnDVCeE1EW52vmO2gmJ9JqQ49TW4IXODWVDCVWPQVLSCqyfE_rYU_SpkltOxkVhYLt_YudZAFrm6hS-XYSNMkLkbG9urdg_C5ZDdcXiFjv1rqutewb7HHaKbxMir2cBjWs7GO2vNztEpB3nKl1ukFVAciGgKTCqpA71pmx6p05mA6b-jIWyaErg9F6wOR2lz3dIHc7XNPN4fkoqxTOEUjkbsbSstN66Nrz6SicN7fsfe-_P3HkVXCGM2KDxmA3wARZlfJMzUNdP8YjvowoW2PoW7iUZ8wvEF-i0WGw2SsrGIXOiFNIHJHRE9S9zRSlKuq6rtb7FRB9QjD8PkpwhJ4ceb7vH_PCy3P95n3UHbfk9hu59dtz_93kKxV5wNCQjD_NeDNrXEL2ioTGPi4tU3LaoRAL3S5eqfmgx2wx0VTDG8r9B7TmFYkkdE-h70ZBOYBmZuDYoWAky1NuajiCNZDI7-oJrWE9F0F_-N03efdavHvN6QGEILkgjBRCRnCwmBzXwr-2QQhAyGvPN88yqqAhRrGgGeho4gnNcz2IU14926tndYBaD9huz8_58_7KPDDtLsa6I4_2lVpHVxfSgSlpSROY5ADeUVVY-46oHQIpeUzNBPqU3nlVlP_PyYZpz-227Mja5SuWLSsqb3wa8wUk0bK3Ie1rTxlsDNfCZvS2PD4Hpv8y9BW2ssY5Z05GRIZpAt8ppuE-OmpNdJ_U0IQscZGaI-0FoLhQfLZhsTNQoiBnTpHrzl593DiDJU4ljOaY_c15tgPpGw8Xd-WnkPkiMhBn8Oz86wyuOhf9bsf96Lndvuu5ff_M2TgDz-1ddK763qXvel237131X86cn4bUvbi67PU6Xt_z3Y--e9ntv_wHEyE3kA?type=png)](https://mermaid.live/edit#pako:eNqtV1Fv4jgQ_itWXu6l7SZ0gQatTsqGdhddCxVQrXSqFJnEgLWJnbOd7bGl__3GTgA3JGyvWh7ajPPNZ898nrHz7MQ8Ic7AIWJI8Urg7JEh-AVhOHkYz9FzaeqfVIKyFcJxzAumIpqg-7_QoxOUNhoNH50jcEL1Q2zQNxo9rAZq8AQrUv45EIaCYEU5Q0MYb-BeCvJPQVi8AZ-RlAVmMUE3u8Gdw0v5L7wdXTeGE6eUWNGExmxa3YIKtY5YkS2IAOBnbbYt7S1hVwsbjmb3k9loPpqMG1YHRPl-bUCSc0lNThqzfYjlpjWWYxVvfqmi2uRamMkTI-LDgySiFsNkOryeNqyei4SI_fIn2voNq1lg9j1SHGBTEtPcxPgZxk7w1tAVve2wTDlWCGdmWkg1WVBFEhRkdWDF_D2Sm2zBU8De402mWcM1FjhWRNCfZt_WkjSfBuNZELYIrQRmcp-qubaAq1XptyXMriqbsmXXVjJD3SVUffhG1ToR-AmnDVCeE1EW52vmO2gmJ9JqQ49TW4IXODWVDCVWPQVLSCqyfE_rYU_SpkltOxkVhYLt_YudZAFrm6hS-XYSNMkLkbG9urdg_C5ZDdcXiFjv1rqutewb7HHaKbxMir2cBjWs7GO2vNztEpB3nKl1ukFVAciGgKTCqpA71pmx6p05mA6b-jIWyaErg9F6wOR2lz3dIHc7XNPN4fkoqxTOEUjkbsbSstN66Nrz6SicN7fsfe-_P3HkVXCGM2KDxmA3wARZlfJMzUNdP8YjvowoW2PoW7iUZ8wvEF-i0WGw2SsrGIXOiFNIHJHRE9S9zRSlKuq6rtb7FRB9QjD8PkpwhJ4ceb7vH_PCy3P95n3UHbfk9hu59dtz_93kKxV5wNCQjD_NeDNrXEL2ioTGPi4tU3LaoRAL3S5eqfmgx2wx0VTDG8r9B7TmFYkkdE-h70ZBOYBmZuDYoWAky1NuajiCNZDI7-oJrWE9F0F_-N03efdavHvN6QGEILkgjBRCRnCwmBzXwr-2QQhAyGvPN88yqqAhRrGgGeho4gnNcz2IU14926tndYBaD9huz8_58_7KPDDtLsa6I4_2lVpHVxfSgSlpSROY5ADeUVVY-46oHQIpeUzNBPqU3nlVlP_PyYZpz-227Mja5SuWLSsqb3wa8wUk0bK3Ie1rTxlsDNfCZvS2PD4Hpv8y9BW2ssY5Z05GRIZpAt8ppuE-OmpNdJ_U0IQscZGaI-0FoLhQfLZhsTNQoiBnTpHrzl593DiDJU4ljOaY_c15tgPpGw8Xd-WnkPkiMhBn8Oz86wyuOhf9bsf96Lndvuu5ff_M2TgDz-1ddK763qXvel237131X86cn4bUvbi67PU6Xt_z3Y--e9ntv_wHEyE3kA)

This ERD shows how the data appears in the dataset:

[![](https://mermaid.ink/img/pako:eNqtV99P2zAQ_lesvOyFbjCJSq2mSSHlRzRoUVq0F6TITdzWIrEz2xnqgP99ZydNTeIUhOgD5JzvPvvufJ-dJy_hKfHGHhETitcC5_cMwc8PgtnddIGeKlP_pBKUrRFOEl4yFdMU3f5C955f2Sic3HsdcEr1Q2LQFxo9qQda8BQrUv3ZEwaCYEU5QxMYd3CvBPlTEpZswSeUssQsIehiN7hzeKn-BdfhuTOcJKPEiiYwpmt1SyrUJmZlviQCgGfa7Fvae8KuFzYJ57ezebgIZ1PH6oCoaNYGJAWX1OTEme19LBe9sXSrePFmFdW20IWZPTIivt1JIloxzKLJeeRYPRcpEc3yZ9r6hNUsMXuIFQdYRBJamBjPYOwAbwtd09sOq4xjhXBupoVUkyVVJEV-3gbWzA-x3OZLngH2Fm9zzRpssMCJIoL-M_u2laRF5E_nftBTaCUwk02qFtoCrt5Kvy9hdlfZlD27ti4z9F1K1bffVG1SgR9x5oDygoiqOV8z34CYHEirDe2mtgIvcWY6GVqsfvJXkFRk-R6uhz1JX01a28lUUSjY3m_sJAvY2kR1la9nvqu8EBlrqnsNxmeV1XBdQsR6t7br2sq-wXbTTuFlWjblNKhJbXfZimq3S0DecKY22RbVDSAdAUmFVSl3rHNjtZXZjyYuXcYi3asyGL0HTGGr7GGB3O1wTbeA505WKZwjkMjdjJVlp3Wv2osoDBZuyW60__bAkVfDGc6JDZqC7YAJsq7KE5mHdv0Yj_kqpmyDQbdwVZ4p_4r4CoX7QbdXXjIKyogzSByR8SP0vc0UZyo-PT7W9X4FRD8QDH-MEhxBk-OT0WjU5YWXA_3mY9TfjyvukZNbvx2MPky-VvEJMDiS8dOMu1mTCtJUJDB2t7VMy2mHUiy1XLyq5p0es4uJIg13tPtfkOY1iSWop9B3I78aQHMz0HUoGcmLjJsejmENJB6d6gmtYT0XQV9Gp-_yHvZ4D93pAYQghSCMlELGcLCYHLfCP7dBCEDopD_fPM-pAkGME0FzqKOJJzDP7SAOeQ1tr6GlAC0NeH4eDPhTc2UeG7lLsFbksOnUNrq-kI5NS0uawiR78I6qxtp3RO3gS8kTaibQp_TOq6bUTs_P73WyYbWnUWTtcoVlz4qqG5_GXEJJdNn7kPa1pwo2gWuhEw1Tm-NzbPSXoSvYyhrnHXk5ETmmKXynGMG999SGaJ3U0JSscJmZI-0FoLhUfL5liTdWoiRHXlloZa8_brzxCmcSRvUVh4ub6tsn4WxF197LfwBRISI?type=png)](https://mermaid.live/edit#pako:eNqtV99P2zAQ_lesvOyFbjCJSq2mSSHlRzRoUVq0F6TITdzWIrEz2xnqgP99ZydNTeIUhOgD5JzvPvvufJ-dJy_hKfHGHhETitcC5_cMwc8PgtnddIGeKlP_pBKUrRFOEl4yFdMU3f5C955f2Sic3HsdcEr1Q2LQFxo9qQda8BQrUv3ZEwaCYEU5QxMYd3CvBPlTEpZswSeUssQsIehiN7hzeKn-BdfhuTOcJKPEiiYwpmt1SyrUJmZlviQCgGfa7Fvae8KuFzYJ57ezebgIZ1PH6oCoaNYGJAWX1OTEme19LBe9sXSrePFmFdW20IWZPTIivt1JIloxzKLJeeRYPRcpEc3yZ9r6hNUsMXuIFQdYRBJamBjPYOwAbwtd09sOq4xjhXBupoVUkyVVJEV-3gbWzA-x3OZLngH2Fm9zzRpssMCJIoL-M_u2laRF5E_nftBTaCUwk02qFtoCrt5Kvy9hdlfZlD27ti4z9F1K1bffVG1SgR9x5oDygoiqOV8z34CYHEirDe2mtgIvcWY6GVqsfvJXkFRk-R6uhz1JX01a28lUUSjY3m_sJAvY2kR1la9nvqu8EBlrqnsNxmeV1XBdQsR6t7br2sq-wXbTTuFlWjblNKhJbXfZimq3S0DecKY22RbVDSAdAUmFVSl3rHNjtZXZjyYuXcYi3asyGL0HTGGr7GGB3O1wTbeA505WKZwjkMjdjJVlp3Wv2osoDBZuyW60__bAkVfDGc6JDZqC7YAJsq7KE5mHdv0Yj_kqpmyDQbdwVZ4p_4r4CoX7QbdXXjIKyogzSByR8SP0vc0UZyo-PT7W9X4FRD8QDH-MEhxBk-OT0WjU5YWXA_3mY9TfjyvukZNbvx2MPky-VvEJMDiS8dOMu1mTCtJUJDB2t7VMy2mHUiy1XLyq5p0es4uJIg13tPtfkOY1iSWop9B3I78aQHMz0HUoGcmLjJsejmENJB6d6gmtYT0XQV9Gp-_yHvZ4D93pAYQghSCMlELGcLCYHLfCP7dBCEDopD_fPM-pAkGME0FzqKOJJzDP7SAOeQ1tr6GlAC0NeH4eDPhTc2UeG7lLsFbksOnUNrq-kI5NS0uawiR78I6qxtp3RO3gS8kTaibQp_TOq6bUTs_P73WyYbWnUWTtcoVlz4qqG5_GXEJJdNn7kPa1pwo2gWuhEw1Tm-NzbPSXoSvYyhrnHXk5ETmmKXynGMG999SGaJ3U0JSscJmZI-0FoLhUfL5liTdWoiRHXlloZa8_brzxCmcSRvUVh4ub6tsn4WxF197LfwBRISI)

In [ ]:
# Verify 1:1 relationships between CLIENT, LOAN and DISPOSITION
assert dispositions_df['client_id'].is_unique, "Each client_id should appear exactly once in the DISPOSITION DataFrame."
assert loans_df['account_id'].is_unique, "Each account_id should appear exactly once in the LOAN DataFrame."

# Verify 1:M relationships between ACCOUNT and DISPOSITION
#assert dispositions['account_id'].is_unique == False, "An account_id should appear more than once in the DISPOSITION DataFrame."
assert dispositions_df['account_id'].is_unique == True, "An account_id should appear once in the DISPOSITION DataFrame."
# TODO check if in accordance to decision to remove disponents from dispositions

# Verify each district_id in ACCOUNT and CLIENT exists in DISTRICT
assert set(accounts_df['district_id']).issubset(
    set(districts_df['district_id'])), "All district_ids in ACCOUNT should exist in DISTRICT."
assert set(clients_df['district_id']).issubset(
    set(districts_df['district_id'])), "All district_ids in CLIENT should exist in DISTRICT."

# Verify each account_id in DISPOSITION, ORDER, TRANSACTION, and LOAN exists in ACCOUNT
assert set(dispositions_df['account_id']).issubset(
    set(accounts_df['account_id'])), "All account_ids in DISPOSITION should exist in ACCOUNT."
assert set(orders_df['account_id']).issubset(
    set(accounts_df['account_id'])), "All account_ids in ORDER should exist in ACCOUNT."
assert set(transactions_df['account_id']).issubset(
    set(accounts_df['account_id'])), "All account_ids in TRANSACTION should exist in ACCOUNT."
assert set(loans_df['account_id']).issubset(
    set(accounts_df['account_id'])), "All account_ids in LOAN should exist in ACCOUNT."

# Verify each client_id in DISPOSITION exists in CLIENT
assert set(dispositions_df['client_id']).issubset(
    set(clients_df['client_id'])), "All client_ids in DISPOSITION should exist in CLIENT."

# Verify each disp_id in CARD exists in DISPOSITION
assert set(cards_df['disp_id']).issubset(set(dispositions_df['disp_id'])), "All disp_ids in CARD should exist in DISPOSITION."

# Non-transactional Data

In [ ]:
orders_pivot_df = orders_df.pivot_table(index='account_id',
                                        columns='k_symbol',
                                        values='debited_amount',
                                        aggfunc='sum',
                                        fill_value=0)

orders_pivot_df.columns = [f'k_symbol_debited_sum_{col.lower()}' for col in orders_pivot_df.columns]

# TODO: find something better than this
orders_pivot_df = orders_pivot_df.reset_index() # Use created index as account_id
orders_pivot_df.head()

In [ ]:
def merge_non_transactional_data(clients, districts, dispositions, accounts, orders, loans, cards):
    # Rename district_id for clarity in clients and accounts DataFrames
    clients = clients.rename(columns={'district_id': 'client_district_id'})
    accounts = accounts.rename(columns={'district_id': 'account_district_id'})
    
    # Prepare districts dataframe for merge with prefix for clients and accounts
    districts_client_prefixed = districts.add_prefix('client_')
    districts_account_prefixed = districts.add_prefix('account_')
    
    # Merge district information for clients and accounts with prefixed columns
    clients_with_districts = pd.merge(clients, districts_client_prefixed, left_on='client_district_id', right_on='client_district_id', how='left')
    accounts_with_districts = pd.merge(accounts, districts_account_prefixed, left_on='account_district_id', right_on='account_district_id', how='left')

    # Merge cards with dispositions and prefix card-related columns to avoid confusion
    cards_prefixed = cards.add_prefix('card_')
    dispositions_with_cards = pd.merge(dispositions, cards_prefixed, left_on='disp_id', right_on='card_disp_id', how='left')
    
    # Merge clients (with district info) with dispositions and cards
    # Assuming dispositions might have columns that overlap with clients, prefix those if necessary
    clients_dispositions_cards = pd.merge(dispositions_with_cards, clients_with_districts, on='client_id', how='left')
    
    # Merge the above with accounts (with district info) on account_id
    accounts_clients_cards = pd.merge(accounts_with_districts, clients_dispositions_cards, on='account_id', how='left')
    
    # Merge orders DataFrame, assuming orders might contain columns that could overlap, prefix as needed
    orders_prefixed = orders.add_prefix('order_')
    comprehensive_df_with_orders = pd.merge(accounts_clients_cards, orders_prefixed, left_on='account_id', right_on='order_account_id', how='left')
    
    # Merge loans with the comprehensive dataframe (now including orders) on account_id
    # Prefix loan-related columns to maintain clarity
    loans_prefixed = loans.add_prefix('loan_')
    final_df = pd.merge(comprehensive_df_with_orders, loans_prefixed, left_on='account_id', right_on='loan_account_id', how='left')

    final_df['account_created'] = pd.to_datetime(final_df['account_created'])
    final_df['card_issued'] = pd.to_datetime(final_df['card_issued'])
    final_df['has_card'] = final_df['card_issued'].notna()
    return final_df

non_transactional_df = merge_non_transactional_data(clients_df, districts_df, dispositions_df, accounts_df, orders_pivot_df, loans_df, cards_df)
non_transactional_df.info()

In [ ]:
non_transactional_df.to_csv("./data/non_transactional.csv")

## EDA

### Card Holders

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Number of Clients by Card Type')
sns.barplot(x=['No Card', 'Classic/Gold Card Holders', 'Junior Card Holders'], y=[non_transactional_df['card_type'].isna().sum(), non_transactional_df['card_type'].isin(['gold', 'classic']).sum(), non_transactional_df['card_type'].eq('junior').sum()])
# ensure that the number of clients is shown on the bars
for i, v in enumerate([non_transactional_df['card_type'].isna().sum(), non_transactional_df['card_type'].isin(['gold', 'classic']).sum(), non_transactional_df['card_type'].eq('junior').sum()]):
    plt.text(i, v + 10, str(v), ha='center', va='bottom')

plt.show()

Looking at the distribution of card holders in general we can see that the most clients are not in a possession of a credit card. 


In [ ]:
plt.figure(figsize=(10, 6))
plt.title(f'Distribution of Age for Junior Card Holders\n total count = {len(non_transactional_df[non_transactional_df["card_type"] == "junior"])}')
sns.histplot(non_transactional_df[non_transactional_df['card_type'] == 'junior']['age'], kde=True, bins=30)
plt.xlabel('Age of Client (presumably in 1999)')
plt.show()

Looking at the age distribution of Junior Card holders paints a picture on this group, however only looking at the current age may be misleading as we need to understand how old they were when the card was issued to determine if they could have been eligble for a Classic/Gold card (at least 18 when the card was issued). 

In [ ]:
non_transactional_df['card_issued'] = pd.to_datetime(non_transactional_df['card_issued'])

non_transactional_df['age_at_card_issuance'] = non_transactional_df['card_issued'] - non_transactional_df['birth_date']
non_transactional_df['age_at_card_issuance'] = non_transactional_df['age_at_card_issuance'].dt.days // 365

plt.figure(figsize=(10, 6))
plt.title(f'Distribution of Age at Card Issuance for Junior Card Holders\n total count = {len(non_transactional_df[non_transactional_df["card_type"] == "junior"])}')
sns.histplot(non_transactional_df[non_transactional_df['card_type'] == 'junior']['age_at_card_issuance'], kde=True, bins=30)
plt.xlabel('Age at Card Issuance')
plt.show()

Here we can see that roughly 1/3 of the Junior Card holders were not of legal age (assuming legal age is 18) when receiving their Junior Card.

In [ ]:
plt.figure(figsize=(10, 6))
plt.title(f'Distribution of Age at Card Issuance for All Card Types\n total count = {len(non_transactional_df)}')
sns.histplot(non_transactional_df[non_transactional_df['card_type'] == 'junior']['age_at_card_issuance'], kde=True, bins=10, color='blue', label='Junior Card Holders')
sns.histplot(non_transactional_df[non_transactional_df['card_type'] != 'junior']['age_at_card_issuance'], kde=True, bins=30, color='red', label='Non-Junior Card Holders')
plt.legend()
plt.xlabel('Age at Card Issuance')
plt.show()

Comparing the age at issue date between Junior and non-Junior (Classic/Gold) card holders shows that there is no overlap between the two groups, which makes intutively sense. 

Therefore removing the subset of Junior Cards seems as valid as there is no reason to believe that there are Junior Cards issued wrongly, the subset being relatively small compared to the remaining issued cards and the fact that our target is specifically Classic/Gold Card owners.  

In [ ]:
before_len = len(non_transactional_df)
non_transactional_df = non_transactional_df[non_transactional_df['card_type'] != 'junior']
data_reduction["Junior Card Holders"] = -(before_len - len(non_transactional_df))
del before_len

Looking at the age distribution of Junior card holders and their occurence in comparison it seems valid to remove them as they are not the target group and make up a small subset of the complete dataset.

### Time factors on Card Status

The time between creating an account and issuing a card may also be important when filtering customers based on their history. We should avoid filtering out potentially interesting periods and understand how the timespans between account creation and card issuance are distributed.

In [ ]:
non_transactional_w_cards_df = non_transactional_df[non_transactional_df['card_issued'].notna() & non_transactional_df['account_created'].notna()]
non_transactional_w_cards_df['duration_days'] = (non_transactional_w_cards_df['card_issued'] - non_transactional_w_cards_df['account_created']).dt.days

plt.figure(figsize=(12, 8))
sns.histplot(non_transactional_w_cards_df['duration_days'], bins=50, edgecolor='black', kde=True)
plt.title('Distribution of Duration Between Account Creation and Card Issuance')
plt.xlabel('Duration in Days')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

The histogram displays a distribution with multiple peaks, indicating that there are several typical time frames for card issuance after account creation. The highest peak occurs within the first 250 days, suggesting that a significant number of cards are issued during this period. The frequency decreases as duration increases, with noticeable peaks that may correspond to specific processing batch cycles or policy changes over time. The distribution also has a long tail, suggesting that in some cases, card issuance can take a very long time.

Analyzing the length of time a client has been with the bank in relation to their account creation date and card ownership can provide valuable insights for a bank's customer relationship management and product targeting strategies. Long-standing clients may exhibit different banking behaviors, such as product adoption and loyalty patterns, compared to newer clients.

In [ ]:
max_account_creation_date = non_transactional_df['card_issued'].max()

non_transactional_df['client_tenure_years_relative'] = (max_account_creation_date - non_transactional_df['account_created']).dt.days / 365.25

plt.figure(figsize=(10, 6))
ax = sns.histplot(
    data=non_transactional_df, 
    x='client_tenure_years_relative', 
    hue='has_card', 
    multiple='stack', 
    binwidth=1,
    stat="percent"
)

# Call the function to add labels
add_percentage_labels(ax, non_transactional_df['has_card'].unique())

# Additional plot formatting
plt.title('Client Tenure Relative to Latest Card Issued Date and Card Ownership')
plt.xlabel('Client Tenure (Years, Relative to Latest Card Issuance)')
plt.ylabel('Percentage of Clients')
plt.tight_layout()

# Display the plot
plt.show()

The bar chart shows the tenure of clients in years, categorized by whether they own a credit card (True) or not (False). Each bar represents the percentage of clients within a specific tenure range, allowing for comparison of the distribution of card ownership among clients with different lengths of association with the bank.



### Demographics

Using the available demographic data, we can investigate the potential correlation between demographic data and card status. The average salary may indicate a difference between cardholders and non-cardholders, as it is reasonable to assume that cardholders have a higher average salary than non-cardholders.

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='has_card', y='client_average_salary', data=non_transactional_df)
plt.title("Average Salary in Client's Region by Card Ownership")
plt.xlabel('Has Card')
plt.ylabel('Average Salary')
plt.xticks([0, 1], ['No Card Owner', 'Card Owner'])

plt.tight_layout()
plt.show()


The box plot compares the average salaries of clients who own a credit card with those who do not. Both groups have a substantial overlap in salary ranges, suggesting that while there might be a trend for card owners to have higher salaries, the difference is not significant. The median salary for card owners is slightly higher than that for non-card owners, as indicated by the median line within the respective boxes.

Both distributions have outliers on the higher end, indicating that some individuals have salaries significantly above the average in both groups. However, these outliers do not dominate the general trend. 

It should also be noted that this plot assumes that the average salary of the region's clients remained constant over the years, which is unlikely to be true.

The group of bar charts represents the distribution of credit card ownership across various demographics, showing the percentage of clients with and without cards within different age groups, sexes, and regions.

In [ ]:
non_transactional_df['age_group'] = pd.cut(non_transactional_df['age'], bins=[0, 25, 40, 55, 70, 100], labels=['<25', '25-40', '40-55', '55-70', '>70'])

plt.figure(figsize=(15, 15))

# Age Group
plt.subplot(3, 1, 1)
age_group_counts = non_transactional_df.groupby(['age_group', 'has_card']).size().unstack(fill_value=0)
age_group_percentages = (age_group_counts.T / age_group_counts.sum(axis=1)).T * 100
age_group_plot = age_group_percentages.plot(kind='bar', stacked=True, ax=plt.gca())
age_group_plot.set_title('Card Ownership by Age Group')
age_group_plot.set_ylabel('Percentage')
add_percentage_labels(age_group_plot, non_transactional_df['has_card'].unique())

# Sex
plt.subplot(3, 1, 2)
sex_counts = non_transactional_df.groupby(['sex', 'has_card']).size().unstack(fill_value=0)
sex_percentages = (sex_counts.T / sex_counts.sum(axis=1)).T * 100
sex_plot = sex_percentages.plot(kind='bar', stacked=True, ax=plt.gca())
sex_plot.set_title('Card Ownership by Sex')
sex_plot.set_ylabel('Percentage')
add_percentage_labels(sex_plot, non_transactional_df['has_card'].unique())

# Client Region
plt.subplot(3, 1, 3)
region_counts = non_transactional_df.groupby(['client_region', 'has_card']).size().unstack(fill_value=0)
region_percentages = (region_counts.T / region_counts.sum(axis=1)).T * 100
region_plot = region_percentages.plot(kind='bar', stacked=True, ax=plt.gca())
region_plot.set_title('Card Ownership by Client Region')
region_plot.set_ylabel('Percentage')
region_plot.tick_params(axis='x', rotation=45)
add_percentage_labels(region_plot, non_transactional_df['has_card'].unique())

plt.tight_layout()
plt.show()


**Card Ownership by Age Group:**
The bar chart displays the proportion of cardholders in different age groups. The percentage of cardholders is lowest in the age group of over 70, followed by the age group of 55-70, indicating that card ownership is more prevalent among younger demographics.

**Card Ownership by Sex:**
The bar chart shows the breakdown of card ownership by sex. The data reveals that the percentage of cardholders is comparable between both sexes, and no significant difference is present.

**Card Ownership by Region**
The bar chart at the bottom illustrates card ownership across different regions, showing a relatively consistent pattern among most regions.

### Impact of Loans / Debt

In [ ]:
simplified_loan_status_mapping = {
    "Contract finished, no problems": "Finished",
    "Contract finished, loan not paid": "Not Paid",
    "Contract running, OK thus-far": "Running",
    "Contract running, client in debt": "In Debt",
    "No Loan": "No Loan"
}

non_transactional_df['loan_status_simplified'] = non_transactional_df['loan_status'].map(simplified_loan_status_mapping)

## this variable wants to kill itself
loan_status_simplified_card_ownership_counts = non_transactional_df.groupby(['loan_status_simplified', 'has_card']).size().unstack(fill_value=0)
loan_status_simplified_card_ownership_percentages = (loan_status_simplified_card_ownership_counts.T / loan_status_simplified_card_ownership_counts.sum(axis=1)).T * 100

loan_status_simplified_card_ownership_percentages.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.title('Interaction Between Simplified Loan Status and Card Ownership')
plt.xlabel('Simplified Loan Status')
plt.ylabel('Percentage of Clients')
plt.xticks(rotation=45)
plt.legend(title='Has Card', labels=['No Card', 'Has Card'])
plt.tight_layout()
plt.show()


# Transactional Data

## Set artificial issue date for non-card holders

In [ ]:
def add_months_since_account_to_card(df):
    df['months_since_account_to_card'] = df.apply(
        lambda row: (row['card_issued'].to_period('M') - row['account_created'].to_period('M')).n
        if pd.notnull(row['card_issued']) and pd.notnull(row['account_created']) else np.nan, axis=1)
    return df

def filter_clients_without_sufficient_history(non_transactional_df, min_history_months=25):
    if 'months_since_account_to_card' not in non_transactional_df.columns:
        print("Warning: months_since_account_to_card column not found. Calculating history length.")
        non_transactional_df = add_months_since_account_to_card(non_transactional_df)

    count_before = len(non_transactional_df)
    filtered_df = non_transactional_df[non_transactional_df['months_since_account_to_card'].isnull() | (non_transactional_df['months_since_account_to_card'] >= min_history_months)]
    print(f"Filtered out {count_before - len(filtered_df)} records with less than {min_history_months} months of history. Percentage: {(count_before - len(filtered_df)) / count_before * 100:.2f}%.")
    return filtered_df

before_len = len(non_transactional_df)
non_transactional_w_sufficient_history_df = filter_clients_without_sufficient_history(non_transactional_df)
data_reduction["Clients without sufficient history"] = -(before_len - len(non_transactional_w_sufficient_history_df))
del before_len

In [ ]:
non_transactional_w_card_df = non_transactional_w_sufficient_history_df.dropna(subset=['card_issued']).copy()

plt.figure(figsize=(12, 8))
sns.histplot(non_transactional_w_card_df['months_since_account_to_card'], kde=True, bins=30)
plt.title('Distribution of Months from Account Creation to Card Issuance')
plt.xlabel('Months')
plt.ylabel('Count')
plt.grid(True)
plt.tight_layout()
plt.show()

### Match by similar transaction activity

The following approaches are in discussion:
- Looking at the distributions above extract the amount of history a buyer most likely has at the issue data of the card
- For each non buyer, find a buyer which was active in a similar time window (Jaccard similarity on the Year-Month sets)

#### NEW IDEA

Instead of looking at the full activity of a buyer, we only look at the pre-purchase activity as there is reason to believe that clients may change their patterns after purchasing date and therefore add unwanted bias. 

![](./docs/IMG_BBEF82A6C6B5-1.jpeg)

### Focus on Contextual Engagement

Our approach to setting artificial card issue date for non-cardholders goes beyond simple feature matching. We concentrate on the periods during which clients engage with the bank in the form of transactions. This method assumes that clients active during similar periods might be influenced by the same economic and societal conditions, providing a more nuanced foundation for establishing connections between current cardholders and potential new ones.

The process emphasizes matching based on the timing of activity, rather than a wide array of characteristics. By identifying when both existing cardholders and non-cardholders interacted with the bank, we can infer a level of behavioral alignment that extends beyond mere transactional data. This alignment suggests a shared response to external conditions.

### Mechanism of Matching

Our matching utilizes the Jaccard similarity index to compare activity patterns:

- **Activity Comparison**: We compare a vector representing an existing cardholder's monthly activity against a matrix of non-cardholders' activity patterns. Here we only consider the activity from the first transaction period across all customers to the card issue date.
- **Use of Jaccard Index**: This metric helps us quantify the resemblance between the activity patterns, focusing on the occurrence of activity rather than its volume, to identify meaningful parallels.

### Eligibility and Selection

Potential matches are subjected to specific criteria to ensure alignment for later model construction:

1. **Account History**: Non-cardholders must have an established history of interaction, with at least 25 months of history between account creation and card issuance (12 months = period as a new customer + 13 months = one year of history + 1 month lag).
2. **Period of Activity**: The analysis considers activities up to the card issuance date of the cardholder, aligning potential non-cardholders based on their response to similar conditions.
3. **Selection Process**: From the non-cardholders with high similarity scores, one is randomly selected from the top-N candidates to maintain fairness in the matching process.

### Objective of the Process

This matching strategy is designed to identify potential cardholders whose historical interaction patterns suggest a fit with card holders. The focus on specific periods of engagement, backed by eligibility criteria and a fair selection process, ensures that new additions to our cardholder base share a commonality that goes beyond surface-level similarities without searching a direct twin for each card holder.

### Construction of the Activity Matrix

The activity matrix serves as the foundation of our matching process, mapping out the engagement of clients with our services over time. It is constructed from transaction data, organizing client interactions into a structured format that highlights periods of activity.

1. **Data Aggregation**: We start with transaction data, which records each client's interactions across various months. This data includes every transaction made by both current cardholders and potential non-cardholders.

2. **Temporal Transformation**: Each transaction is associated with a specific date. These dates are then transformed into monthly periods, consolidating daily transactions into a monthly view of activity. This step simplifies the data, focusing on the presence of activity within each month rather than the specific dates or frequencies of transactions.

3. **Matrix Structure**: The transformed data is arranged into a matrix format. Rows represent individual clients, identified by their account IDs. Columns correspond to monthly periods, spanning the entire range of months covered by the transaction data.

4. **Activity Indication**: In the matrix, a cell value is set to indicate the presence of activity for a given client in a given month. If a client made one or more transactions in a month, the corresponding cell is marked to reflect this activity. The absence of transactions for a client in a month leaves the cell unmarked.

5. **Binary Representation**: The final step involves converting the activity indicators into a binary format. Active months are represented by a '1', indicating the presence of transactions, while inactive months are denoted by a '0', indicating no transactions.

The heatmap provided offers a visual representation of the activity matrix for clients, depicting the levels of engagement over various periods.

- **Diagonal Trend**: There is a distinct diagonal pattern, indicating that newer accounts (those created more recently) have fewer periods of activity. This makes sense as these accounts have not had the opportunity to transact over the earlier periods displayed on the heatmap.
  
- **Darker Areas (Purple)**: These represent periods of inactivity where clients did not engage. The darker the shade, the less activity occurred in that particular period for the corresponding set of accounts.
  
- **Brighter Areas (Yellow)**: In contrast, the brighter areas denote periods of activity. A brighter shade implies more clients were active during that period.
  
- **Account Creation Date**: Clients are sorted by their account creation date. Those who joined earlier are at the top, while more recent clients appear toward the bottom of the heatmap.

In [ ]:
def prepare_activity_matrix(transactions):
    """
    Create an activity matrix from transaction data.

    The function transforms transaction data into a binary matrix that indicates
    whether an account was active in a given month.

    Parameters:
    - transactions (pd.DataFrame): A DataFrame containing the transaction data.

    Returns:
    - pd.DataFrame: An activity matrix with accounts as rows and months as columns.
    """
    transactions['month_year'] = transactions['date'].dt.to_period('M')
    transactions['active'] = 1
    
    activity_matrix = transactions.pivot_table(index='account_id', 
                                    columns='month_year', 
                                    values='active', 
                                    fill_value=0)
    
    activity_matrix.columns = [f'active_{str(col)}' for col in activity_matrix.columns]
    return activity_matrix

def plot_activity_matrix(activity_matrix):
    sparse_matrix = activity_matrix.astype(bool)    
    plt.figure(figsize=(20, 10))
    sns.heatmap(sparse_matrix, cmap='viridis', cbar=True, yticklabels=False)
    plt.title(f'Activity Matrix across all clients sorted by account creation date')
    plt.xlabel('Period')
    plt.ylabel('Accounts')
    plt.show()

activity_matrix = prepare_activity_matrix(transactions_df)
plot_activity_matrix(activity_matrix)

In [ ]:
from sklearn.metrics import pairwise_distances
from tqdm import tqdm

def check_eligibility_for_matching(non_cardholder, cardholder, verbose=False):
    """
    Determine if a non-cardholder is eligible for matching with a cardholder.

    This function checks whether the card issuance to a cardholder occurred at least
    25 months after the non-cardholder's account was created.

    Parameters:
    - non_cardholder (pd.Series): A data series containing the non-cardholder's details.
    - cardholder (pd.Series): A data series containing the cardholder's details.
    - verbose (bool): If True, print detailed eligibility information. Default is False.

    Returns:
    - bool: True if the non-cardholder is eligible for matching, False otherwise.
    """
    if cardholder['card_issued'] <= non_cardholder['account_created']:
        return False    

    period_diff = (cardholder['card_issued'].to_period('M') - non_cardholder['account_created'].to_period('M')).n

    if verbose:
        print(f"Card issued: {cardholder['card_issued']}, Account created: {non_cardholder['account_created']}, Period diff: {period_diff}, Eligible: {period_diff >= 25}")

    return period_diff >= 25

def match_cardholders_with_non_cardholders(non_transactional, transactions, top_n=5):
    """
    Match cardholders with non-cardholders based on the similarity of their activity patterns.

    The function creates an activity matrix, identifies eligible non-cardholders, calculates
    the Jaccard similarity to find matches, and randomly selects one match per cardholder
    from the top N similar non-cardholders.

    Parameters:
    - non_transactional (pd.DataFrame): A DataFrame containing non-cardholders.
    - transactions (pd.DataFrame): A DataFrame containing transactional data.
    - top_n (int): The number of top similar non-cardholders to consider for matching.

    Returns:
    - list: A list of tuples with the cardholder and matched non-cardholder client IDs and similarity scores.
    """
    with_card = non_transactional[non_transactional['card_issued'].notna()]
    without_card = non_transactional[non_transactional['card_issued'].isna()]

    activity_matrix = prepare_activity_matrix(transactions)
    
    with_card_activity = with_card.join(activity_matrix, on='account_id', how='left')
    without_card_activity = without_card.join(activity_matrix, on='account_id', how='left')

    matched_non_cardholders = set()
    matches = []

    for idx, cardholder in tqdm(with_card_activity.iterrows(), total=len(with_card_activity), desc='Matching cardholders'):
        issue_period = cardholder['card_issued'].to_period('M')
        eligible_cols = [col for col in activity_matrix if col.startswith('active') and pd.Period(col.split('_')[1]) <= issue_period]

        if not eligible_cols:
            print(f"No eligible months found for cardholder client_id {cardholder['client_id']}.")
            continue

        cardholder_vector = cardholder[eligible_cols].fillna(0).astype(bool).values.reshape(1, -1)
        non_cardholder__matrix = without_card_activity[eligible_cols].fillna(0).astype(bool).values        
        assert cardholder_vector.shape[1] == non_cardholder__matrix.shape[1], "Dimension mismatch between cardholder and applicant activity matrix."

        distances = pairwise_distances(cardholder_vector, non_cardholder__matrix, metric='jaccard').flatten()
        eligible_non_cardholders = [i for i, applicant in without_card_activity.iterrows()
                                    if check_eligibility_for_matching(applicant, cardholder) and i not in matched_non_cardholders]

        if eligible_non_cardholders:
            select_non_cardholders(distances, eligible_non_cardholders, matches, matched_non_cardholders, cardholder, without_card_activity, top_n)
        else:
            print(f"No eligible non-cardholders found for cardholder client_id {cardholder['client_id']}.")
            
    return matches

def select_non_cardholders(distances, eligible_non_cardholders, matches, matched_applicants, cardholder, without_card_activity, top_n):
    """
    Randomly select a non-cardholder match for a cardholder from the top N eligible candidates.

    Parameters:
    - distances (np.array): An array of Jaccard distances between a cardholder and non-cardholders.
    - eligible_non_cardholders (list): A list of indices for non-cardholders who are eligible for matching.
    - matches (list): A list to which the match will be appended.
    - matched_applicants (set): A set of indices for non-cardholders who have already been matched.
    - cardholder (pd.Series): The data series of the current cardholder.
    - without_card_activity (pd.DataFrame): A DataFrame of non-cardholders without card issuance.
    - top_n (int): The number of top similar non-cardholders to consider for matching.

    Returns:
    - None: The matches list is updated in place with the selected match.
    """
    eligible_distances = distances[eligible_non_cardholders]
    sorted_indices = np.argsort(eligible_distances)[:top_n]

    if sorted_indices.size > 0:
        selected_index = np.random.choice(sorted_indices)
        actual_selected_index = eligible_non_cardholders[selected_index]

        if actual_selected_index not in matched_applicants:
            matched_applicants.add(actual_selected_index)
            applicant = without_card_activity.iloc[actual_selected_index]
            similarity = 1 - eligible_distances[selected_index]
            
            matches.append((cardholder['client_id'], applicant['client_id'], similarity))

def set_artificial_issue_dates(non_transactional_df, matches):
    """
    Augment the non-transactional DataFrame with artificial card issue dates based on matching results.

    Each matched non-cardholder is assigned a card issue date corresponding to their matched
    cardholder. The 'has_card' flag for each non-cardholder is updated accordingly.

    Parameters:
    - non_transactional_df (pd.DataFrame): The DataFrame of non-cardholders to augment.
    - matches (list): A list of tuples containing the matched cardholder and non-cardholder IDs and similarity scores.

    Returns:
    - pd.DataFrame: The augmented DataFrame with artificial card issue dates.
    """
    augmented_df = non_transactional_df.copy()
    augmented_df['has_card'] = True

    for cardholder_id, non_cardholder_id, _ in matches:
        card_issue_date = augmented_df.loc[augmented_df['client_id'] == cardholder_id, 'card_issued'].values[0]
        augmented_df.loc[augmented_df['client_id'] == non_cardholder_id, ['card_issued', 'has_card']] = [card_issue_date, False]

    return augmented_df

matched_non_card_holders_df = match_cardholders_with_non_cardholders(non_transactional_w_sufficient_history_df, transactions_df)

print(f"Percentage of clients with card issued: {non_transactional_w_sufficient_history_df['card_issued'].notna().mean() * 100:.2f}%")
matched_non_card_holders_w_issue_date_df = set_artificial_issue_dates(non_transactional_w_sufficient_history_df, matched_non_card_holders_df)
print(f"Percentage of clients with card issued after matching: {matched_non_card_holders_w_issue_date_df['card_issued'].notna().mean() * 100:.2f}%")

After each non-cardholder got the artifical card issued date assigned we drop the remaining non-cardholders without a match.

In [ ]:
before_len = len(matched_non_card_holders_w_issue_date_df)
print(-(before_len - len(matched_non_card_holders_w_issue_date_df)))
matched_non_card_holders_w_issue_date_df = matched_non_card_holders_w_issue_date_df.dropna(subset=['card_issued'])
data_reduction["Non-cardholders without match"] = -(before_len - len(matched_non_card_holders_w_issue_date_df))
del before_len

## Aggregate Transactions on a Monthly Basis

Validating first transactions where the amount equals the balance is essential for the integrity of our aggregated data analysis. This specific assertion underpins the reliability of our subsequent aggregation operations by ensuring each account's financial history starts from a verifiable point.

In [ ]:
zero_amount_transactions_df = transactions_df[transactions_df['amount'] == 0]

zero_amount_transactions_info = {
    'total_zero_amount_transactions': len(zero_amount_transactions_df),
    'unique_accounts_with_zero_amount': zero_amount_transactions_df['account_id'].nunique(),
    'transaction_type_distribution': zero_amount_transactions_df['transaction_type'].value_counts(normalize=True),
    'operation_distribution': zero_amount_transactions_df['operation'].value_counts(normalize=True),
    'k_symbol_distribution': zero_amount_transactions_df['k_symbol'].value_counts(normalize=True)
}

zero_amount_transactions_info, len(zero_amount_transactions_info)

In [ ]:
accounts_with_zero_amount_transactions = accounts_df[accounts_df['account_id'].isin(zero_amount_transactions_df['account_id'].unique())]
accounts_with_zero_amount_transactions

In [ ]:
del accounts_with_zero_amount_transactions 
del zero_amount_transactions_df
del zero_amount_transactions_info

In [ ]:
def validate_first_transactions(transactions):
    """
    Validates that for each account in the transactions DataFrame, there is at least
    one transaction where the amount equals the balance on the account's first transaction date.

    Parameters:
    - transactions (pd.DataFrame): DataFrame containing transaction data with columns
      'account_id', 'date', 'amount', and 'balance'.

    Raises:
    - AssertionError: If not every account has a first transaction where the amount equals the balance.
    """

    first_dates = transactions.groupby('account_id')['date'].min().reset_index(name='first_date')

    first_trans = pd.merge(transactions, first_dates, how='left', on=['account_id'])

    first_trans_filtered = first_trans[(first_trans['date'] == first_trans['first_date']) & (first_trans['amount'] == first_trans['balance'])]

    first_trans_filtered = first_trans_filtered.drop_duplicates(subset=['account_id'])

    unique_accounts = transactions['account_id'].nunique()
    assert unique_accounts == first_trans_filtered['account_id'].nunique(), "Not every account has a first transaction where the amount equals the balance."

    return "Validation successful: Each account has a first transaction where the amount equals the balance."

validate_first_transactions(transactions_df)

We can confirm the truth of the assertions made. It is certain that there is a transaction with an amount equal to the balance in the transaction history of any account on the first date.

The function `aggregate_transactions_monthly` is designed to process and summarize financial transactions on a monthly basis for each account within a dataset. The explanation of its workings, step by step, is as follows:

1. **Sorting Transactions**: Initially, the function sorts the transactions in the provided DataFrame `transactions_df` based on `account_id` and the transaction `date`. This ensures that all transactions for a given account are ordered chronologically, which is crucial for accurate monthly aggregation and cumulative balance calculation.

2. **Monthly Grouping**: Each transaction's date is then converted to a monthly period using `dt.to_period("M")`. This step categorizes each transaction by the month and year it occurred, facilitating the aggregation of transactions on a monthly basis.

3. **Aggregation of Monthly Data**: The function groups the sorted transactions by `account_id` and the newly created `month` column. For each group, it calculates several metrics:
   - `volume`: The sum of all transactions' amounts for the month, representing the total money flow.
   - `total_abs_amount`: The sum of the absolute values of the transactions' amounts, indicating the total amount of money moved, disregarding the direction.
   - `transaction_count`: The count of transactions, providing a sense of activity level.
   - `positive_transaction_count` and `negative_transaction_count`: The counts of positive (inflows) and negative (outflows) transactions, respectively. This distinction can help identify the balance between income and expenses.
   - Statistical measures like `average_amount`, `median_amount`, `min_amount`, `max_amount`, and `std_amount` offer insights into the distribution of transaction amounts.
   - `type_count`, `operation_count`, and `k_symbol_count`: The counts of unique transaction types, operations, and transaction symbols (k_symbol), respectively, indicating the diversity of transaction characteristics.

4. **Cumulative Balance Calculation**: After aggregating the monthly data, the function computes a cumulative balance (`balance`) for each account by cumulatively summing the `volume` (total transaction amount) over time. This step provides insight into how the account balance evolves over the months.

5. **Validation of Aggregated Data**: The `validate_monthly_aggregated_transactions` function is invoked to ensure the integrity and correctness of the aggregated data through several assertions:
   - The balance should consistently increase or decrease based on whether the total monthly transaction volume is positive or negative, respectively.
   - For each account, the balance in the first month should equal the total transaction volume of that month.
   - The sum of positive and negative transaction counts must equal the total transaction count for each month.
   - The number of unique accounts in the aggregated data should match that in the original dataset.
   - The final balances of accounts in the aggregated data should closely match their last recorded transactions in the original dataset.

In [ ]:
def aggregate_transactions_monthly(df):
    """
    Aggregate financial transaction data on a monthly basis per account.

    Parameters:
    - df (pd.DataFrame): DataFrame containing financial transaction data with 'account_id', 'date', and other relevant columns.

    Returns:
    - pd.DataFrame: Monthly aggregated financial transaction data per account.
    """
    df_sorted = df.sort_values(by=["account_id", "date"])
    df_sorted["month"] = df_sorted["date"].dt.to_period("M")

    monthly_aggregated_data = (
            df_sorted.groupby(["account_id", "month"])
            .agg(
                volume=("amount", "sum"),
                total_abs_amount=("amount", lambda x: x.abs().sum()),
                transaction_count=("amount", "count"),
                positive_transaction_count=("amount", lambda x: (x >= 0).sum()), # TODO: it seems that there are some transactions with 0 amount, how to handle those?
                negative_transaction_count=("amount", lambda x: (x < 0).sum()),
                average_amount=("amount", "mean"),
                median_amount=("amount", "median"),
                min_amount=("amount", "min"),
                max_amount=("amount", "max"),
                std_amount=("amount", "std"),
                type_count=("transaction_type", "nunique"),
                operation_count=("operation", "nunique"),
                k_symbol_count=("k_symbol", "nunique"),
            )
            .reset_index()
            .sort_values(by=["account_id", "month"])
        )

    monthly_aggregated_data["balance"] = monthly_aggregated_data.groupby("account_id")["volume"].cumsum()

    validate_monthly_aggregated_transactions(monthly_aggregated_data, df)

    return monthly_aggregated_data

def validate_monthly_aggregated_transactions(aggregated_data, original_df):
    """
    Validate the integrity and correctness of aggregated monthly financial transactions.

    Parameters:
    - aggregated_data (pd.DataFrame): Aggregated monthly transaction data.
    - original_df (pd.DataFrame): Original dataset of financial transactions.

    Raises:
    - AssertionError: If validation conditions are not met.
    """
    
    assert (aggregated_data["volume"] >= 0).all() == (
        aggregated_data["balance"].diff() >= 0
    ).all(), "If the total amount is positive, the balance should go up."

    assert (aggregated_data["volume"] < 0).all() == (
        aggregated_data["balance"].diff() < 0
    ).all(), "If the total amount is negative, the balance should go down."

    first_month = aggregated_data.groupby("account_id").nth(0)
    assert (
        first_month["volume"] == first_month["balance"]
    ).all(), "The balance should equal the volume for the first month."

    assert (
        aggregated_data["positive_transaction_count"]
        + aggregated_data["negative_transaction_count"]
        == aggregated_data["transaction_count"]
    ).all(), "The sum of positive and negative transaction counts should equal the total transaction count."
    
    assert (
        aggregated_data["account_id"].nunique() == original_df["account_id"].nunique()
    ), "The number of unique account_ids in the aggregated DataFrame should be the same as the original DataFrame."

    assert (
        pd.merge(
            aggregated_data.groupby("account_id")
            .last()
            .reset_index()[["account_id", "balance"]],
            original_df[original_df.groupby("account_id")["date"].transform("max") == original_df["date"]][
                ["account_id", "balance"]
            ],
            on="account_id",
            suffixes=("_final", "_last"),
        )
        .apply(
            lambda x: np.isclose(x["balance_final"], x["balance_last"], atol=5), axis=1
        )
        .any()
    ), "Some accounts' final balances do not match their last transactions."
    

agg_transactions_monthly_df = aggregate_transactions_monthly(transactions_df)
validate_monthly_aggregated_transactions(agg_transactions_monthly_df, transactions_df)
agg_transactions_monthly_df.describe()

### Monthly Balance Difference and Volume

This plot gives a clear picture of how money moves in and out of an account each month and how these movements affect the overall balance. It does this by showing two things:

- **Balance Difference**: This line shows whether the account balance went up or down each month. If the line goes up, it means the account gained money that month. If it goes down, the account lost money.
- **Volume**: This line shows the total amount of money that moved in the account each month, regardless of whether it was coming in or going out.

**What to Look For**:
- A direct link between the amount of money moved (volume) and changes in the account balance. High incoming money should lead to an uptick in the balance, and lots of outgoing money should lead to a downturn.
- This visual check helps to understand how active the account is and whether it’s generally getting fuller or emptier over time.

In [ ]:
def plot_monthly_balance_diff_and_volume(transactions_monthly, account_id, figsize=(12, 8)):
    account_transactions = transactions_monthly[transactions_monthly['account_id'] == account_id].sort_values(by='month')
    account_transactions['balance_diff'] = account_transactions['balance'].diff()

    plt.figure(figsize=figsize)

    plt.plot(account_transactions['month'].astype(str), account_transactions['balance_diff'], marker='o', label='Balance Difference')
    plt.plot(account_transactions['month'].astype(str), account_transactions['volume'], marker='x', linestyle='--', label='Volume')

    plt.title(f'Monthly Balance Difference and Volume for Account {account_id}')
    plt.xlabel('Month')
    plt.ylabel('Value')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_monthly_balance_diff_and_volume(agg_transactions_monthly_df, 2)

### Monthly Transactions, Balance, and Volume Plot Explanation

This visualization offers a snapshot of an account’s activity over time by comparing money movement each month with the overall account balance. It helps to understand:

- **Volume**: How much money came in or went out of the account each month. Incoming money is shown as up, and outgoing money as down.
- **Balance**: The total money in the account at the end of each month, showing how it's changed over time due to the monthly transactions.

**What to Look For**:
- How the monthly money movement impacts the account's growing or shrinking balance. For example, a few months of high income should visibly increase the balance.
- This simple visual guide helps spot trends, like if the account is steadily growing, holding steady, or facing issues, giving quick insights into financial well-being and further validates the aggregation made in the previous step.

In [ ]:
def plot_monthly_transactions_balance_and_volume(agg_transactions_monthly, account_id):
    account_transactions = agg_transactions_monthly[agg_transactions_monthly['account_id'] == account_id]

    plt.figure(figsize=(15, 10))

    plt.plot(account_transactions['month'].astype(str), account_transactions['volume'], marker='o', label='Volume')
    plt.plot(account_transactions['month'].astype(str), account_transactions['balance'], marker='x', linestyle='--', label='Balance')

    plt.title(f'Monthly Transactions and Balance for Account {account_id}')
    plt.xlabel('Month')
    plt.ylabel('Value')
    plt.xticks(rotation=60)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_monthly_transactions_balance_and_volume(agg_transactions_monthly_df, 2)

### Delieverable: Closer Look at account 14 and 18

#### Account 14 

In [ ]:
plot_monthly_transactions_balance_and_volume(agg_transactions_monthly_df, 14)

Account 14 shows a rather conservative transaction history. The spending habits are all withing range of 10k to -10k per month. We can see little volatility, the account shows a slight trend of growing.

#### Account 18

In [ ]:
plot_monthly_transactions_balance_and_volume(agg_transactions_monthly_df, 18)

Account 18 paints a different picture in comparison to account 14. 

The volatility here is a lot higher, indiciating a potential for a business account or high income household. Especially March 1994 to December 1994 show some volatile transaction habits.

##### Analysis
Looking at the balance and volume per month for the accounts 14 and 18 we can notice some interesting patterns.

###### Account 14:

## Rolling Window Aggregations for Card Holders

We condense transaction data into a monthly aggregated format. This aggregation serves a multifaceted purpose:

- Monthly aggregation standardizes the time frame across which we analyze transactions, allowing us to compare transactional behaviors consistently across all accounts.
- Aggregating data on a monthly level illuminates patterns that daily data might obscure. It enables us to discern trends over a broader time scale, capturing cyclical behaviors, seasonal effects, and response to macroeconomic events.
- Daily transaction data can be "noisy" with random fluctuations. By considering monthly totals and averages, we reduce this noise, revealing underlying trends more clearly.
- Our primary objective is to understand behaviors leading up to the issuance of a card. Aggregating transactions on a monthly basis helps focus on the crucial period preceding card issuance, enabling us to correlate transactional behaviors with the propensity to become a cardholder.

In [ ]:
def pivot_transactions(non_transactional, transactions_monthly, months_before_card_range=(2, 13)):
    """
    Aggregate monthly transaction data and merge it with non-transactional account data, 
    focusing on the time frame leading up to the card issuance.

    This function merges monthly transaction data with non-transactional data to associate each
    transaction with the respective account and card issued date. It then filters transactions based
    on a specified range of months before card issuance and aggregates various transaction metrics.

    Parameters:
    - non_transactional (pd.DataFrame): A DataFrame containing non-transactional account data.
    - transactions_monthly (pd.DataFrame): A DataFrame containing monthly transaction data.
    - months_before_card_range (tuple): A tuple specifying the inclusive range of months before card 
                                        issuance to filter the transactions for aggregation.

    The aggregation includes the sum of volume and transaction counts, as well as the mean and other
    statistical measures of transaction amounts, for each account within the specified months before 
    card issuance.

    The resulting DataFrame is pivoted to have 'account_id' as rows and the months before card 
    issuance as columns, with aggregated metrics as values. Column names are constructed to 
    describe the month and the metric represented.

    Returns:
    - pd.DataFrame: The final aggregated and pivoted dataset ready for analysis, with each row 
                    representing an account and each column a specific metric in the months before 
                    card issuance.
    """
    merged_df = transactions_monthly.merge(non_transactional[['account_id']], on='account_id')

    merged_df['card_issued_date'] = merged_df['account_id'].map(non_transactional.set_index('account_id')['card_issued'])
    merged_df['months_before_card'] = merged_df.apply(lambda row: (row['card_issued_date'].to_period('M') - row['month']).n, axis=1)

    start_month, end_month = months_before_card_range
    filtered_df = merged_df.query(f"{start_month} <= months_before_card <= {end_month}")
    
    aggregated_data = filtered_df.groupby(['account_id', 'months_before_card']).agg({
        'volume': 'sum',
        'total_abs_amount': 'sum',
        'transaction_count': 'sum',
        'positive_transaction_count': 'sum',
        'negative_transaction_count': 'sum',
        'average_amount': 'mean',
        'median_amount': 'median',
        'min_amount': 'min',
        'max_amount': 'max',
        'std_amount': 'std',
        'type_count': 'sum',
        'operation_count': 'sum',
        'k_symbol_count': 'sum',
        'balance': 'mean'
    }).reset_index()

    pivoted_data = aggregated_data.pivot(index='account_id', columns='months_before_card')
    pivoted_data.columns = ['_'.join(['M', str(col[1]), col[0]]) for col in pivoted_data.columns.values]

    final_dataset = pivoted_data.reset_index()
    return final_dataset

transactions_pivoted_df = pivot_transactions(matched_non_card_holders_w_issue_date_df, agg_transactions_monthly_df)
transactions_pivoted_df.describe()

In [ ]:
matched_non_card_holders_w_issue_date_df

In [ ]:
agg_transactions_monthly_df.to_csv("./data/transactions_monthly.csv")

# Merge everything together

In [ ]:
golden_record_df = matched_non_card_holders_w_issue_date_df.merge(transactions_pivoted_df, on='account_id', how='left')
assert golden_record_df['client_id'].is_unique, "Each client_id should appear exactly once in the final DataFrame."
assert golden_record_df['account_id'].is_unique, "Each account_id should appear exactly once in the final DataFrame."

In [ ]:
golden_record_df.to_csv("data/bronze_record.csv")

In [ ]:
golden_record_df.describe()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Number of Clients by Card Issuance Status')
sns.countplot(x='has_card', data=golden_record_df)
plt.xlabel('Card Issued')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Distribution of Card Issuance Dates')
sns.histplot(golden_record_df, x='card_issued', hue='has_card', kde=True, bins=30)
plt.xlabel('Card Issuance Date')
plt.ylabel('Count')
plt.show()

In [ ]:
data_reduction

In [ ]:
# Adjusting the y-axis range to extend up to 5000

fig, ax = plt.subplots(figsize=(10, 6))

# Plot bars
bars = ax.bar(adjusted_df.index, adjusted_df['amount'], bottom=[0, *bottoms[1:]])

# Adjust bar colors and heights for clarity
bars[0].set_color('skyblue')  # Start bar color
bars[-1].set_color('skyblue')  # Net bar color
for i, bar in enumerate(bars[1:-1], start=1):
    if adjusted_df.iloc[i]['amount'] < 0:
        bar.set_color('orange')

# Add amount labels to bars
for bar, value in zip(bars, adjusted_df['amount']):
    height = bar.get_height()
    ax.annotate(f'{value:,.0f}',
                xy=(bar.get_x() + bar.get_width() / 2, bar.get_y() + height/2),
                xytext=(0, 3),  # 3 points vertical offset
                textcoords="offset points",
                ha='center', va='bottom')

# Connect bars with lines
for i in range(len(bars) - 1):  # Exclude connecting the last net bar
    ax.plot([bars[i].get_x() + bars[i].get_width()/2, bars[i+1].get_x() + bars[i+1].get_width()/2],
            [bars[i].get_y() + bars[i].get_height(), bars[i].get_y() + bars[i].get_height()], 'k-')

ax.axhline(y=0, color='k', linewidth=1)
ax.set_ylim(0, 5000)  # Extend y-axis up to 5000
ax.set_title('Data Reduction Waterfall with Adjusted Y-Axis and Labels')
ax.set_xticklabels(adjusted_df.index, rotation=45)
ax.set_ylabel('Amount')

plt.tight_layout()
plt.show()



In [ ]:
# Correcting the "Net" bar representation in the waterfall chart

fig, ax = plt.subplots(figsize=(10, 6))

# Recalculate the 'Net' value explicitly excluding the last adjustment
net_value = sum(data_reduction.values())

# Plot the bars, excluding the 'Net' calculation for visual adjustment
bars = ax.bar(adjusted_df.index[:-1], adjusted_df['amount'][:-1], bottom=[0, *bottoms[1:-1]])

# Add the 'Net' bar, which should visually indicate the final state without extending upwards
bars_net = ax.bar('Net', net_value, bottom=0)
bars_net[0].set_color('skyblue')  # Set 'Net' bar color

# Adjust colors for other bars
for bar, value in zip(bars, adjusted_df['amount'][:-1]):  # Exclude 'Net' from loop
    if value < 0:
        bar.set_color('orange')
    else:
        bar.set_color('skyblue')

# Label bars with their amounts, adjusting positioning for readability
for bar, value in zip(bars, adjusted_df['amount'][:-1]):  # Exclude 'Net' from labels
    ax.annotate(f'{value:,.0f}',
                xy=(bar.get_x() + bar.get_width() / 2, bar.get_y() + (bar.get_height() if bar.get_height() > 0 else 0)),
                xytext=(0, 3),  # 3 points vertical offset
                textcoords="offset points",
                ha='center', va='bottom')

# Special handling for 'Net' bar label
ax.annotate(f'{net_value:,.0f}',
            xy=(bars_net[0].get_x() + bars_net[0].get_width() / 2, net_value/2),  # Position label in the middle of 'Net' bar
            xytext=(0, 3),  # 3 points vertical offset
            textcoords="offset points",
            ha='center', va='bottom')

# Set the chart title, labels, and adjust y-axis limit
ax.axhline(y=0, color='k', linewidth=1)
ax.set_ylim(0, 5000)  # Set y-axis limit to include up to 5000 for visual clarity
ax.set_title('Data Reduction Waterfall: Final Net Representation')
ax.set_xticklabels(adjusted_df.index, rotation=45)
ax.set_ylabel('Amount')

plt.tight_layout()
plt.show()


# Model Construction

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

def train_evaluate_model(golden_record_df, feature_columns, model, target_column='has_card', cv=10):
    """
    Trains and evaluates a given model based on specified feature columns using cross-validation.
    
    Parameters:
    - golden_record_df: DataFrame containing the Golden Record dataset.
    - feature_columns: List of column names to be used as features.
    - model: The machine learning model to be trained and evaluated.
    - target_column: Name of the target column.

    Returns:
    - metrics_summary: Summary of evaluation metrics including mean and standard deviation for accuracy, F1, AUC-ROC, precision, recall.
    """
    X = golden_record_df[feature_columns]
    y = golden_record_df[target_column]

    numerical_features = [col for col in feature_columns if golden_record_df[col].dtype in ['int64', 'float64']]
    categorical_features = [col for col in feature_columns if golden_record_df[col].dtype == 'object']

    numerical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    categorical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_pipeline, numerical_features),
            ('cat', categorical_pipeline, categorical_features)
        ])

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    scoring = {
        'accuracy': 'accuracy',
        'f1': make_scorer(f1_score),
        'roc_auc': 'roc_auc',
        'precision': make_scorer(precision_score),
        'recall': make_scorer(recall_score)
    }

    metrics_summary = cross_validate(pipeline, X, y, scoring=scoring, cv=cv, return_train_score=False)

    metrics_report = {}
    for metric in scoring.keys():
        metrics_report[metric] = {
            'mean': metrics_summary[f'test_{metric}'].mean(),
            'std': metrics_summary[f'test_{metric}'].std()
        }
        print(f"{metric}: mean = {metrics_report[metric]['mean']:.2f}, std = {metrics_report[metric]['std']:.2f}")

    return metrics_report

In [ ]:
import pandas as pd

def pretty_print_metrics(metrics_summary):
    """
    Pretty prints the evaluation metrics as a pandas DataFrame.
    
    Parameters:
    - metrics_summary: Dictionary containing the metrics summary.

    Returns:
    - A pandas DataFrame with the metrics summary.
    """ 
    df = pd.DataFrame(columns=['Metric', 'Type', 'Mean Score', 'Std Dev'])

    for metric, values in metrics_summary.items():
        df = pd.concat(df, pd.DataFrame({
            'Metric': metric,
            'Mean Score': values.get('mean', 0),
            'Std Dev': values.get('std', 0),
        }), ignore_index=True)

    return df

metrics_summary = {
    'accuracy': {'mean': 0.7339044, 'std': 0.014434028},
    'f1': {'mean': 0.7238686, 'std': 0.017242956},
    'precision': {'mean': 0.7325809, 'std': 0.017027547},
    'recall': {'mean': 0.7218968, 'std': 0.029497014},
    'roc_auc': {'mean': 0.8053633, 'std': 0.007226673}
}

df_metrics = pretty_print_metrics(metrics_summary)
print(df_metrics)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Define the logistic regression model
model = LogisticRegression(random_state=42, max_iter=1000)

# Define the feature columns as specified earlier or based on new requirements
feature_columns = [
    'age',  # Placeholder, replace with the actual column name for age if exists
    'client_region'  # Placeholder, adjust based on actual dataset structure
] + [col for col in golden_record_df.columns if 'M_' in col and ('_balance' in col or '_volume' in col)]

# Assuming golden_record_df is already loaded and available
# Call the function with the logistic regression model and the specified feature set
train_evaluate_model(golden_record_df, feature_columns, model)

# Feature Engineering

# Model Engineering

# Model Comparison & Selection

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Define your model
model = RandomForestClassifier(random_state=42)

# Call the function with your model and feature set
train_evaluate_model(golden_record_df, feature_columns, model)


# WORKBENCH - remove later

### Some older input

We need some categorical indicator wheter a transactions is a transactions incoming or outgoing from the perspective of the account holder. This will be important for the feature engineering later on. We will create a column called `transaction_direction` using the amount to engineer this feature.

Balance is the wealth on the account after the transaction.

k_symbol is the purpose of the transaction. This is often use in the context of budgeting in E-Banking applications or just personal finance management. A lot of NA values are present in this column. We will have to deal with this later on and weigh the importance of this column.

Track the time series of a given account to get a better understanding of the datasets nature.

It seems that there can be multiple transactions on the same day. We will have to aggregate the transactions on the same day to get a better understanding of the transactions as the timestamp resolution is not high enough to track the transactions on a daily basis.

We need some handling for this as the ID is not informative as well (Dani). 

For the feature enginnering a per month evaluation of the transactions is sufficient (Dani). 

We need to make sure across the board that for the prediction we only use the data that is available at the time of the prediction. This means that we can only use the data from the past to predict the future. This is important to keep in mind when we engineer the features as some entities do not have any information about the date and therefore we cannot use them for the prediction as we cannot rule out that they are not from the future.

Frequency analysis of the transactions could be interesting as the hypothesis might be that the more frequent the transactions the more likely the account holder is to be interested in a credit card. Fourier transformation could be used to get a better understanding of the frequency of the transactions.

### JITT 05.03.24
- New customers are handled differently
- Customer without the required history should be ignored otherwise they are treated as irrelevant
- Lag is ignored like (12 + 1) months
- Age should be in relation to the time of the event (card issued / reference date for refrence clients)
- How old are customers with a Junior Card? This should be evaluated based on the data
    - Example with Junior Card model with Age as most important feature as a negative example
- Reference clients
    - They should not be as similar as possible (Twin brother problem)
    - Same external market conditions
    - Same environment
    - See slide 6
- Owner and disponents cannot be distinguished directly and assumptions are required
    - MasterCards vs Visa war: as much cards as possible for both client of an account
    - AGain the Twin brother problem as features are too similar possibly

#### General notes to self
- Visualise monthly product puchases
- Viz environment of selected client and reference clients and answer the questions are they from a comparable environment

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7d865ccc-7b5c-4f8d-b4e6-4e008791345d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>